In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#libraries
!pip install -q transformers accelerate evaluate bert-score sacremoses sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login

In [ ]:
#add your Hugging Face token inside the quotes
login("your_huggingface_token_here")


model_id = "google/gemma-2b"  #model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# loading dataset from drive
def load_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

complex_sents = load_file("/content/drive/MyDrive/Sinhala-simp-complex/Data/complex-1000.txt")
simple1_sents = load_file("/content/drive/MyDrive/Sinhala-simp-complex/Data/simp1000-1.txt")
simple2_sents = load_file("/content/drive/MyDrive/Sinhala-simp-complex/Data/simp1000-2.txt")
simple3_sents = load_file("/content/drive/MyDrive/Sinhala-simp-complex/Data/simp1000-3.txt")

assert len(complex_sents) == len(simple1_sents) == len(simple2_sents) == len(simple3_sents)

In [ ]:
# prompt
preds = []
for i, sentence in enumerate(complex_sents):
    prompt = f"Simplify the following Sinhala sentence.\n{sentence}\n"
    output = pipe(prompt, max_new_tokens=512, do_sample=False)[0]['generated_text']
    simplified = output.split('\n')[-1].strip()
    preds.append(simplified)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
#reference structure
refs_grouped = [[simple1_sents[i], simple2_sents[i], simple3_sents[i]] for i in range(len(preds))]

In [ ]:
lengths = [len(tokenizer(t)['input_ids']) for t in preds]
print(f"Avg length: {sum(lengths)/len(lengths):.2f}, Max: {max(lengths)}")

Avg length: 103.81, Max: 513


In [ ]:
#SARI
import evaluate
sari = evaluate.load("sari")

sari_scores = [
    sari.compute(
        sources=[complex_sents[i]],
        predictions=[preds[i]],
        references=[refs_grouped[i]]
    )["sari"]
    for i in range(len(preds))
]
avg_sari = sum(sari_scores) / len(sari_scores)
print("✅ SARI (avg):", round(avg_sari, 2))

✅ SARI (avg): 16.99


In [ ]:
!pip install bert_score
from bert_score import score as bert_score_fn

all_precisions = []
all_recalls = []
all_f1s = []

# loop over each prediction
for i in range(len(preds)):
    sample_precisions = []
    sample_recalls = []
    sample_f1s = []

    #compare against each of the 3 references
    for ref in refs_grouped[i]:
        P, R, F1 = bert_score_fn([preds[i]], [ref], lang="si", rescale_with_baseline=True)
        sample_precisions.append(P[0].item())
        sample_recalls.append(R[0].item())
        sample_f1s.append(F1[0].item())

    # take best match per metric
    all_precisions.append(max(sample_precisions))
    all_recalls.append(max(sample_recalls))
    all_f1s.append(max(sample_f1s))

# Compute averages
avg_precision = sum(all_precisions) / len(all_precisions)
avg_recall = sum(all_recalls) / len(all_recalls)
avg_f1 = sum(all_f1s) / len(all_f1s)

# results
print("\n📊 Evaluation Metrics:")
print(f"BERTScore Precision: {avg_precision:.4f}")
print(f"BERTScore Recall:    {avg_recall:.4f}")
print(f"BERTScore F1:        {avg_f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


📊 Evaluation Metrics:
BERTScore Precision: 0.7772
BERTScore Recall:    0.7297
BERTScore F1:        0.7505
